<a href="https://colab.research.google.com/github/veren4/SMILES_featurization/blob/master/My_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Imports

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from torch.autograd import Variable
import os
#import torchvision    # data loaders for common datasets such as Imagenet, FashionMNIST, MNIST, etc. and data transformers for images

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0) # set default size of plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
import platform
print('Using python: ', platform.python_version())
print('Using torch version: ', torch.__version__)
print('Using device: ', device)
# Machine: 2015 13" Macbook Pro, i5 dual core

#import torch.nn as nn
from torch import nn
import torch.nn.functional as F
import timeit

Using python:  3.6.9
Using torch version:  1.6.0+cu101
Using device:  cpu


In [4]:
from google.colab import drive
drive.mount('/content/drive')
import pickle

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
import tensorboard
#print(f"Tensorboard version: {tensorboard.__version__}")

# Load the TensorBoard notebook extension
%load_ext tensorboard.notebook

# imports
%autoreload 2

In [ ]:
from torch.utils.tensorboard import SummaryWriter

In [35]:
import torchvision.transforms as transforms

###Set up TensorBoard

[How to TensorBoard](https://pytorch.org/docs/stable/tensorboard.html)\
[PyTorch Tensorboard tutorial](https://pytorch.org/tutorials/intermediate/tensorboard_tutorial.html)\
([Medium article](https://medium.com/@iamsdt/using-tensorboard-in-google-colab-with-pytorch-458f9bb95212) on TensorBoard in Colab with PyTorch) <- no suitable version available\
[The best tutorial](https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/tensorboard_with_pytorch.ipynb#scrollTo=IRSe6eHcFPyT): in a colab notebook

###Seed

In [15]:
torch.manual_seed(1)

https://discuss.pytorch.org/t/reproducibility-with-all-the-bells-and-whistles/81097

###Description & documentation

Later: automated batching\
Visualization

[FloydHub](https://blog.floydhub.com/long-short-term-memory-from-zero-to-hero-with-pytorch/)\
[GitHub Minimal example](https://github.com/chrisvdweth/ml-toolkit/blob/master/pytorch/notebooks/minimal-example-lstm-input.ipynb)\
[PyTorch LSTM Beginner guide](https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html)\
[PyTorch LSTM Guide](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)\
[Medium article](https://medium.com/@sunitachoudhary103/generating-molecules-using-a-char-rnn-in-pytorch-16885fd9394b)

[deeplearningwizard](https://www.deeplearningwizard.com/deep_learning/practical_pytorch/pytorch_lstm_neuralnetwork/): Steps

Step 1: Load Dataset\
Step 2: Make Dataset Iterable\
Step 3: Create Model Class\
Step 4: Instantiate Model Class\
Step 5: Instantiate Loss Class\
Step 6: Instantiate Optimizer Class\
Step 7: Train Model


##Data preparation (int_tokens)

In [16]:
infile1 = open('/content/drive/My Drive/Rostlab internship/7_PyTorch/Tokenized_Dataset', 'rb')
tokenized_dataset = pickle.load(infile1)
infile1.close()

####Create vocabulary

In [17]:
alphabet = set()

for i in tokenized_dataset:
  alphabet.update(i)

In [18]:
#alphabet    # length: 12

####Token alphabet

In [19]:
dict_token_alphabet = {}
dict_token_alphabet.update({'PAD': 0, 'UNK': 1, 'SOS': 2, 'EOS':3})

index = 4
for i in alphabet:
  dict_token_alphabet.update({i: index})
  index = index+1

dict_token_alphabet

{'(': 6,
 ')': 14,
 '1': 10,
 '2': 11,
 '3': 4,
 '4': 9,
 '=': 12,
 'C': 15,
 'EOS': 3,
 'N': 7,
 'O': 5,
 'P': 13,
 'PAD': 0,
 'S': 8,
 'SOS': 2,
 'UNK': 1}

####Vectorize SMILES

In [47]:
int_tokens = [None]*len(tokenized_dataset)    # empty list of length 14

for i in range(len(tokenized_dataset)):
  int_tokens[i] = [None]*len(tokenized_dataset[i])

  for j in range(len(tokenized_dataset[i])):
    int_tokens[i][j] = dict_token_alphabet.get(tokenized_dataset[i][j])

Still open: What is a batch for me? For now, I will go with 3 SMILES.

In [23]:
batch = [int_tokens[0], int_tokens[1], int_tokens[3]]

pad_to_this = len(max(batch, key=len))    #  length of the longest list in the batch -> All other elements need to be padded to this length

pad_beginning = True

for SMILES in batch:
  while(len(SMILES) < pad_to_this):
    if(pad_beginning == True):
      SMILES.insert(0, 0)
      pad_beginning = !pad_beginning
    else:
      SMILES.append(0)
      pad_beginning = !pad_beginning

In [24]:
# make a numpy array out of it
batch = np.array(batch)

# make a PyTorch tensor
batch = torch.tensor(batch, dtype=torch.long)

print(batch)
print('The shape of batch is:', batch.shape)

tensor([[15,  6, 15,  6, 15,  6, 15,  5, 13,  6, 12,  5, 14,  6,  5, 14,  5, 14,
          5, 14,  5, 14, 15,  6, 12,  5, 14, 15,  6, 12,  5, 14,  5],
        [ 0, 15, 15, 15,  6, 15, 14, 15,  6, 12,  5, 14, 15,  6, 12,  5, 14,  5,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0],
        [15, 10, 12, 15, 15,  6, 12, 15, 15,  6, 12, 15, 10, 14,  5, 14, 15, 15,
         15,  6, 12,  5, 14,  5,  0,  0,  0,  0,  0,  0,  0,  0,  0]])
The shape of batch is: torch.Size([3, 33])


Now we have the first step of having our data in the shape (batch_size, seq_len). to feed it into an LSTM, we still need input_size.

####Embedding layer

Define layer

In [25]:
vocab_size = len(dict_token_alphabet)  # 14
embed_dim = 10 #<- What does the size of the embeddings mean in my case?

word_embedding_layer = nn.Embedding(vocab_size, embed_dim)

Push batch through layer

In [26]:
batch = word_embedding_layer(batch)

print('The shape of batch is:', batch.shape)   # 3, 33, 10
#print()
#print(batch)

The shape of batch is: torch.Size([3, 33, 10])


Now we have want we want: (batch_size, seq_len, input_size)

####Dataloader

PyTorch Dataloader:\
[PyTorch guide to its Dataloader class](https://pytorch.org/docs/stable/data.html)\
[Parameters](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader)

\
(Alternative to the PyTorch function:  Function from the [Medium article](https://medium.com/@sunitachoudhary103/generating-molecules-using-a-char-rnn-in-pytorch-16885fd9394b))

In [46]:
batch_size = 1
n_iters = 20
num_epochs = n_iters / (len(tokenized_dataset) / batch_size)
num_epochs = int(num_epochs)
print(num_epochs, 'epochs')

1 epochs


In [21]:
my_loader = torch.utils.data.DataLoader(dataset=tokenized_dataset,
                                        batch_size=3,
                                        shuffle=True,
                                        sampler=None,       # optional: custom Sampler object
                                        batch_sampler=None, # optional: provide a custom sampler
                                        num_workers=0,      # if positive int => multi-process data loading
                                        collate_fn=None,    # optional: custom collate function
                                        pin_memory=False,   # to speed it up when working on a GPU
                                        drop_last=False,
                                        timeout=0,
                                        worker_init_fn=None)

I have a map-style dataset, at least for now with this testing dataset. However, when I take the huge PubChem dataset, this might change to iterable-style.\
When automatic batching is enabled, collate_fn is called with a list of data samples at each time. It is expected to collate the input samples into a batch for yielding from the data loader iterator.

##Putting my data into a model

In [27]:
batch.size()

torch.Size([3, 33, 10])

In [28]:
# choose the input parameters
input_size = batch.shape[2]
hidden_dim = 32

# define the model
my_LSTM=nn.LSTM(input_size, hidden_dim)

# initialise the lstm
for parameter in my_LSTM.parameters():
    nn.init.normal_(parameter)
    #print(parameter)

In [29]:
# Model architecture visualization
writer = SummaryWriter()    # default `log_dir` is "runs"

writer.add_graph(model=my_LSTM.cpu(),
                 input_to_model=batch,
                 verbose=True)
writer.close()

NameError: ignored

In [ ]:
#%tensorboard — writer

####Instantiate Loss Class

In [ ]:
#import torch.optim as optim

In [30]:
criterion = nn.CrossEntropyLoss()

####Instantiate Optimizer Class

In [31]:
# minibatch SGD
learning_rate = 0.1
optimizer = torch.optim.SGD(my_LSTM.parameters(), lr=learning_rate)  

In [33]:
#for i in range(len(list(my_LSTM.parameters()))):
#    print(list(my_LSTM.parameters())[i].size())

torch.Size([128, 10])
torch.Size([128, 32])
torch.Size([128])
torch.Size([128])


##Train

In [ ]:
lstm_out, hidden = my_LSTM(batch)

print('The shape of lstm_out is:', lstm_out.shape) # (seq_len, batch_size, hidden_dim)
print('The shape of hidden is:', hidden.shape) # (num_layers*num_directions, batch_size, hidden_dim)

In [ ]:
import timeit

In [ ]:
runs=10**4

print("Time Pytorch LSTM {} runs: {:.3f}s".format(runs, timeit.timeit("my_LSTM(batch)", 
                                       setup="from __main__ import my_LSTM, batch", 
                                       number=runs))
     )

Time Pytorch LSTM 10000 runs: 18.690s


[Optuna](https://github.com/optuna/optuna): automated Hyperparameter Optimization

##Visualization

Let's write some stuff to TensorBoard, and log into it to see how things go :)
You can log into TensorBoard by running the command from this exercise folder in Terminal:

```tensorboard --logdir=runs```

For Linux user, you can use open a Terminal and simply run it

For Windows user with Anaconda, you may open an Anaconda Prompt and then run the command. Otherwise use your default setup of running python code in cmd.

Note that before running the command **you may get into root folder of this Notebook first**. Finally from the command line and then navigating to http://localhost:6006. If everything went well you will be presented with the tensorboard stup and after executing the next cell you should see the following images in TensorBoard.


In [ ]:
# write to tensorboard
#writer.add_image('four_mnist_images', img_grid)     # I need to write my data here!

NameError: ignored

In [ ]:
# Model architecture visualization

writer.add_graph(model=word_embedding_layer.cpu(),
                 input_to_model=batch,
                 verbose=True)
writer.close()

Expected tensor for argument #1 'indices' to have scalar type Long; but got torch.FloatTensor instead (while checking arguments for embedding)
Error occurs, No graph saved


RuntimeError: ignored

In [ ]:
# Start TensorBoard within the notebook using magics function
%tensorboard — logdir logs